<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [3]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [4]:
import folium
import pandas as pd

In [5]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [33]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for lat, lng, label in zip(launch_sites_df['Lat'], launch_sites_df['Long'], launch_sites_df['Launch Site']):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(site_map)

    folium.Marker(
        [lat, lng],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{label}</b></div>'
        )
    ).add_to(site_map)
# show map
site_map

In [13]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [22]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [40]:
spacex_df['Launch Site'].value_counts()

Launch Site
CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: count, dtype: int64

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [23]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [25]:
marker_color = []

for i in spacex_df['class']:
    if i == 1:
        marker_color.append('green')
    elif i == 0:
        marker_color.append('red')

spacex_df['marker_color'] = marker_color
spacex_df.head()
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [32]:
site_map.remove_child(marker_cluster)

<class 'AttributeError'>: 'Map' object has no attribute 'remove_child'

In [35]:
# Add marker_cluster to current site_map
marker_cluster = MarkerCluster().add_to(site_map)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']
        ))
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [36]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [37]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [43]:
# find coordinate of the closet coastline
cali_coastline_lat = 34.63651  
cali_coastline_lon = -120.62439
sp_coastline_lat = 28.56313
sp_coastline_lon = -80.56794
ken_coastline_lat = 28.58016
ken_coastline_lon = -80.64244

distance_coastline=[]

for index, record in spacex_df.iterrows():
    if record['Launch Site'] == 'VAFB SLC-4E':
        dist = calculate_distance(record['Lat'], record['Long'], cali_coastline_lat, cali_coastline_lon)
        distance_coastline.append(dist)
    elif record['Launch Site'] == 'KSC LC-39A':
        dist = calculate_distance(record['Lat'], record['Long'], ken_coastline_lat, ken_coastline_lon)
        distance_coastline.append(dist)
    elif record['Launch Site'] == 'CCAFS LC-40' or record['Launch Site'] == 'CCAFS SLC-40':
        dist = calculate_distance(record['Lat'], record['Long'], sp_coastline_lat, sp_coastline_lon)
        distance_coastline.append(dist)

spacex_df['distance_coastline'] = distance_coastline
spacex_df.groupby('Launch Site').head(

,Launch Site,Lat,Long,class,marker_color,distance_coastline
0,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519
1,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519
2,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519
3,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519
4,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519
26,VAFB SLC-4E,34.632834,-120.610745,0,red,1.313964
27,VAFB SLC-4E,34.632834,-120.610745,0,red,1.313964
28,VAFB SLC-4E,34.632834,-120.610745,1,green,1.313964
29,VAFB SLC-4E,34.632834,-120.610745,1,green,1.313964
30,VAFB SLC-4E,34.632834,-120.610745,1,green,1.313964


In [48]:
coastline_lat = []
coastline_long = []

for index, record in spacex_df.iterrows():
    if record['Launch Site'] == 'VAFB SLC-4E':
        coastline_lat.append(cali_coastline_lat)
        coastline_long.append(cali_coastline_lon)
    elif record['Launch Site'] == 'KSC LC-39A':
        coastline_lat.append(ken_coastline_lat)
        coastline_long.append(ken_coastline_lon)
    elif record['Launch Site'] == 'CCAFS LC-40' or record['Launch Site'] == 'CCAFS SLC-40':
        coastline_lat.append(sp_coastline_lat)
        coastline_long.append(sp_coastline_lon)

spacex_df['Coastline_Lat'] = coastline_lat
spacex_df['Coastline_Long'] = coastline_long
spacex_df.groupby('Launch Site').head()

,Launch Site,Lat,Long,class,marker_color,distance_coastline,Coastline_Lat,Coastline_Long
0,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794
1,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794
2,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794
3,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794
4,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794
26,VAFB SLC-4E,34.632834,-120.610745,0,red,1.313964,34.63651,-120.62439
27,VAFB SLC-4E,34.632834,-120.610745,0,red,1.313964,34.63651,-120.62439
28,VAFB SLC-4E,34.632834,-120.610745,1,green,1.313964,34.63651,-120.62439
29,VAFB SLC-4E,34.632834,-120.610745,1,green,1.313964,34.63651,-120.62439
30,VAFB SLC-4E,34.632834,-120.610745,1,green,1.313964,34.63651,-120.62439


In [49]:
spacex_df.shape

(56, 8)

In [56]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
sites = spacex_df.groupby('Launch Site')

for site_name, group in sites:
    # pick the first row for this site
    row = group.iloc[0]

    distance_marker = folium.Marker(
        [row['Coastline_Lat'], row['Coastline_Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' %
                 "{:10.2f} KM".format(row['distance_coastline']),
        )
    )
    site_map.add_child(distance_marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [57]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
for site, group in sites:
    row = group.iloc[0]
    lines=folium.PolyLine(locations=[[row['Coastline_Lat'],row['Coastline_Long']],[row['Lat'],row['Long']]], weight=1)
    site_map.add_child(lines)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [60]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
proxim = [['Launch Base', 'City_Lat', 'City_Long','Railway_Lat', 'Railway_Long', 'Highway_Lat', 'Highway_Long'],
          ['VAFB', 34.63776, -120.45788, 34.63634, -120.62379, 34.67857, -120.57667],
          ['KSC', 28.61135, -80.81474, 28.57363, -80.65422, 28.576, -80.6451],
          ['LC or SLC', 28.3872, -80.60532, 28.5623, -80.58714, 28.56191, -80.57053]]
proxim = pd.DataFrame(proxim[1:], columns=proxim[0])
proxim

,Launch Base,City_Lat,City_Long,Railway_Lat,Railway_Long,Highway_Lat,Highway_Long
0,VAFB,34.63776,-120.45788,34.63634,-120.62379,34.67857,-120.57667
1,KSC,28.61135,-80.81474,28.57363,-80.65422,28.57600,-80.64510
2,LC or SLC,28.38720,-80.60532,28.56230,-80.58714,28.56191,-80.57053


In [72]:
for index, row in spacex_df.iterrows():
    if row['Launch Site'] == 'VAFB SLC-4E':
        base = 'VAFB'
    elif row['Launch Site'] == 'KSC LC-39A':
        base = 'KSC'
    elif row['Launch Site'] in ['CCAFS LC-40', 'CCAFS SLC-40']:
        base = 'LC or SLC'
    else:
        continue

    spacex_df.at[index, 'City_Lat'] = proxim.query("`Launch Base` == @base")['City_Lat'].values[0]
    spacex_df.at[index, 'City_Long'] = proxim.query("`Launch Base` == @base")['City_Long'].values[0]
    spacex_df.at[index, 'Railway_Lat'] = proxim.query("`Launch Base` == @base")['Railway_Lat'].values[0]
    spacex_df.at[index, 'Railway_Long'] = proxim.query("`Launch Base` == @base")['Railway_Long'].values[0]
    spacex_df.at[index, 'Highway_Lat'] = proxim.query("`Launch Base` == @base")['Highway_Lat'].values[0]
    spacex_df.at[index, 'Highway_Long'] = proxim.query("`Launch Base` == @base")['Highway_Long'].values[0]

spacex_df.head()

,Launch Site,Lat,Long,class,marker_color,distance_coastline,Coastline_Lat,Coastline_Long,City_Lat,City_Long,Railway_Lat,Railway_Long,Highway_Lat,Highway_Long
0,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053
1,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053
2,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053
3,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053
4,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053


In [69]:
spacex_df.columns

Index(['Launch Site', 'Lat', 'Long', 'class', 'marker_color',
       'distance_coastline', 'Coastline_Lat', 'Coastline_Long'],
      dtype='object')

In [74]:
city_distance = []
hw_distance = []
rw_distance = []

for row, col in spacex_df.iterrows():
    city_dist = calculate_distance(col['Lat'], col['Long'], col['City_Lat'], col['City_Long'])
    hw_dist = calculate_distance(col['Lat'], col['Long'], col['Highway_Lat'], col['Highway_Long'])
    rw_dist = calculate_distance(col['Lat'], col['Long'], col['Railway_Lat'], col['Railway_Long'])
    city_distance.append(city_dist)
    hw_distance.append(hw_dist)
    rw_distance.append(rw_dist)

spacex_df['City_Distance'] = city_distance
spacex_df['Highway_Distance'] = hw_distance
spacex_df['Railway_Distance'] = rw_distance
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color,distance_coastline,Coastline_Lat,Coastline_Long,City_Lat,City_Long,Railway_Lat,Railway_Long,Highway_Lat,Highway_Long,City_Distance,Highway_Distance,Railway_Distance
0,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053,19.667532,0.668324,0.955781
1,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053,19.667532,0.668324,0.955781
2,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053,19.667532,0.668324,0.955781
3,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053,19.667532,0.668324,0.955781
4,CCAFS LC-40,28.562302,-80.577356,0,red,0.924519,28.56313,-80.56794,28.3872,-80.60532,28.5623,-80.58714,28.56191,-80.57053,19.667532,0.668324,0.955781


In [75]:
sites = spacex_df.groupby('Launch Site')

for site_name, group in sites:
    # pick the first row for this site
    row = group.iloc[0]

    city_marker = folium.Marker(
        [row['City_Lat'], row['City_Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' %
                 "{:10.2f} KM".format(row['City_Distance']),))
    city_lines=folium.PolyLine(locations=[[row['City_Lat'],row['City_Long']],[row['Lat'],row['Long']]], weight=1)
    
    hw_marker = folium.Marker(
        [row['Highway_Lat'], row['Highway_Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' %
                 "{:10.2f} KM".format(row['Highway_Distance']),))
    hw_lines=folium.PolyLine(locations=[[row['Highway_Lat'],row['Highway_Long']],[row['Lat'],row['Long']]], weight=1)

    rw_marker = folium.Marker(
        [row['Railway_Lat'], row['Railway_Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' %
                 "{:10.2f} KM".format(row['Railway_Distance']),))
    rw_lines=folium.PolyLine(locations=[[row['Railway_Lat'],row['Railway_Long']],[row['Lat'],row['Long']]], weight=1)
    
    
    site_map.add_child(city_marker)
    site_map.add_child(city_lines)
    site_map.add_child(rw_marker)
    site_map.add_child(rw_lines)
    site_map.add_child(hw_marker)
    site_map.add_child(hw_lines)
    
    
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
